In [ ]:
# Lab 11 MNIST and Convolutional Neural Network
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

# CNN模型的数学原理解析

在这段代码中，涉及了多个关键的数学原理和概念，主要来自卷积神经网络（CNN）。以下是对每个部分的详细数学原理解释：

## 1. 卷积操作（Convolution）

卷积操作是CNN中最重要的计算过程。卷积的目的是从输入图像中提取局部特征。

### 数学定义：
卷积操作可以表示为：
$$
(I * K)(x, y) = \sum_m \sum_n I(x+m, y+n) K(m, n)
$$
其中，$I$ 是输入图像，$K$ 是卷积核（filter），$x, y$ 是卷积输出的位置。卷积操作的核心思想是将卷积核与图像的每一部分进行“点乘”运算，从而得到图像的局部特征。

### 在代码中的实现：
- `torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)` 实现了卷积操作，其中：
  - `in_channels`: 输入图像的通道数（在这里是 1，表示灰度图像）。
  - `out_channels`: 卷积核的数量（在第一层是 32，第二层是 64），每个卷积核会生成一个特征图。
  - `kernel_size=3`: 卷积核的大小，表示卷积核是 3x3。
  - `stride=1`: 步长，表示卷积核移动的步幅。步长越大，输出特征图的尺寸越小。
  - `padding=1`: 填充，表示在输入图像的边缘添加零，使得输出图像的尺寸不因卷积而缩小。

## 2. ReLU 激活函数（ReLU Activation）

### 数学定义：
ReLU（Rectified Linear Unit）激活函数的数学表达式是：
$$
f(x) = \max(0, x)
$$
这意味着 ReLU 将输入值小于 0 的部分设为 0，其它部分保持不变。

### 作用：
ReLU 激活函数的作用是引入非线性。卷积层的输出是线性的，如果没有激活函数，神经网络只会是一个线性模型，无法拟合复杂的模式。通过使用 ReLU，网络可以捕捉到更复杂的模式。

## 3. 最大池化（Max Pooling）

池化操作的目的是减少图像的空间尺寸（宽度和高度），从而减少计算量、内存占用，并有助于防止过拟合。

### 数学定义：
最大池化的计算过程是在一个固定大小的窗口内取最大值。例如，假设窗口大小为 2x2：
$$
\text{MaxPooling}(I) = \max(I_{1,1}, I_{1,2}, I_{2,1}, I_{2,2})
$$
其中 $I_{i,j}$ 是输入矩阵的元素。

### 在代码中的实现：
- `torch.nn.MaxPool2d(kernel_size=2, stride=2)` 实现了最大池化操作，窗口大小为 2x2，步长为 2。最大池化将图像的每个 2x2 区域压缩为一个值，从而将图像尺寸减半。

## 4. 全连接层（Fully Connected Layer）

全连接层的数学原理是通过加权求和将所有输入特征组合在一起，生成输出。

### 数学定义：
在全连接层中，输入向量 $\mathbf{x}$ 被映射到输出向量 $\mathbf{y}$：
$$
\mathbf{y} = \mathbf{W} \mathbf{x} + \mathbf{b}
$$
其中，$\mathbf{W}$ 是权重矩阵，$\mathbf{x}$ 是输入向量，$\mathbf{b}$ 是偏置。

### 在代码中的实现：
- `self.fc = torch.nn.Linear(7 * 7 * 64, 10)` 实现了全连接层，它接收 7x7x64 的输入（来自卷积层的输出），并生成 10 个输出（对应 10 个类别）。这里的输入维度是通过卷积和池化操作之后的图像尺寸计算出来的。

## 5. Xavier 初始化（Xavier Initialization）

### 数学原理：
Xavier 初始化是一种用于初始化神经网络权重的方法，目的是避免在训练过程中出现梯度消失或梯度爆炸问题。Xavier 初始化假设激活函数是对称的（例如 ReLU）并且均匀分布的。

Xavier 初始化的规则是：
$$
W \sim U\left(-\frac{\sqrt{6}}{\sqrt{n_{in} + n_{out}}}, \frac{\sqrt{6}}{\sqrt{n_{in} + n_{out}}}\right)
$$
其中，$n_{in}$ 和 $n_{out}$ 分别是输入和输出单元的数量。这个初始化方法确保了每一层的输出方差和输入方差大致相等，从而有助于防止训练初期的梯度问题。

### 在代码中的实现：
- `torch.nn.init.xavier_uniform_(self.fc.weight)` 使用 Xavier 初始化方法初始化全连接层的权重，确保训练过程中的梯度流动良好。

## 6. Flatten 操作

在卷积层和全连接层之间，通常需要将多维的张量（例如，7x7x64）展平成一维向量，以便输入到全连接层。

### 数学定义：
Flatten 操作本质上是将一个矩阵按行或列展开成一维向量。
$$
\text{Flatten}(X) = [X_1, X_2, X_3, \dots]
$$

### 在代码中的实现：
- `out = out.view(out.size(0), -1)` 这行代码将张量 `out` 从形状 `(batch_size, 64, 7, 7)` 转换为 `(batch_size, 7*7*64)`，以便将其传递到全连接层。

## 总结

1. **卷积操作**：从图像中提取局部特征。
2. **ReLU 激活函数**：引入非线性。
3. **最大池化**：减小图像尺寸，减少计算复杂度。
4. **全连接层**：将所有提取的特征组合在一起进行分类。
5. **Xavier 初始化**：防止梯度消失或爆炸，确保权重初始化合适。
6. **Flatten**：将二维数据转换为一维，便于输入到全连接层。

这些步骤共同作用，使得 CNN 能够通过训练自动学习如何从图像中提取重要特征并进行分类。


In [ ]:
# instantiate CNN model
model = CNN().to(device)

In [ ]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

In [ ]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())